# Introduction


This is the **Demand-Based Pricing Model**, a more advanced model where we:

• Construct a mathematical demand function using key features:

*   Occupancy rate
*   Queue length
*   Traffic level
*   Special day
*   Vehicle type

• Ensure demand is normalized and price variations are smooth and bounded (e.g.,
not more than 2x or less than 0.5x base).

In [ ]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [ ]:
# Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df.head()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by 'SystemCodeNumber' and the new 'Timestamp' column, and reset the index
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

In [ ]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["SystemCodeNumber", "Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("demand_parking_stream.csv", index=False)

In [ ]:
# Define the schema for the streaming data using Pathway
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [ ]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming

data_2 = pw.demo.replay_csv("demand_parking_stream.csv", schema=ParkingSchema, input_rate=100)

In [ ]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
data_with_time = data_2.with_columns(
    t = data_2.Timestamp.dt.strptime(fmt),
    day = data_2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 2: Making a demand-based pricing function

In [ ]:
import datetime

# Map categorical features & occupancy ratio
@pw.udf(return_type=int)
def map_traffic(level):
    return {'low':1, 'medium':2, 'high':3}.get(level,1)

@pw.udf(return_type=float)
def vehicle_weight(vtype):
    return {'car':1, 'bike':0.5, 'truck':1.5}.get(vtype,1)

data_mapped = data_with_time.with_columns(
    traffic_num = map_traffic(data_with_time.TrafficConditionNearby),
    vehicle_weight = vehicle_weight(data_with_time.VehicleType),
    occupancy_ratio = data_with_time.Occupancy / data_with_time.Capacity,
)

# Define some required UDFs
@pw.udf(return_type=float)
def normalize_demand(d):
    return max(0, min(1, (d + 3) / 6))

@pw.udf(return_type=float)
def bound_price(p, base_price):
    return max(0.5 * base_price, min(2 * base_price, p))

# Define daily tumbling window and compute everything inside
alpha, beta, gamma, delta, epsilon, lambd = 0.6, 0.3, 0.2, 0.5, 0.4, 1
base_price = 10

daily_window = (
    data_mapped.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        avg_occ_ratio = pw.reducers.avg(pw.this.occupancy_ratio),
        avg_queue = pw.reducers.avg(pw.this.QueueLength),
        avg_traffic = pw.reducers.avg(pw.this.traffic_num),
        avg_vehicle_weight = pw.reducers.avg(pw.this.vehicle_weight),
        special_day = pw.reducers.max(pw.this.IsSpecialDay)
    )
    .with_columns(
        demand = (
            alpha * pw.this.avg_occ_ratio
          + beta * pw.this.avg_queue
          - gamma * pw.this.avg_traffic
          + delta * pw.this.special_day
          + epsilon * pw.this.avg_vehicle_weight
        )
    )
    .with_columns(
        # Normalize demand to [0,1]
        norm_demand = normalize_demand(pw.this.demand)
    )
    .with_columns(
        # Compute price: base*(1+λ*norm_demand), bounded
        raw_price = base_price * (1 + lambd * pw.this.norm_demand),
    )
    .with_columns(
        demand_daily_price = bound_price(pw.this.raw_price, base_price)
    )
)

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define Bokeh plotting function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Demand-based Daily Parking Price",
        x_axis_type="datetime"
    )
    fig.line("t", "demand_daily_price", source=source, line_width=2, color="navy")
    fig.circle("t", "demand_daily_price", source=source, size=6, color="red")
    return fig

# Bind Pathway table to Bokeh plot using .plot()
# sorting_col="t" ensures data points appear in time order
viz = daily_window.plot(price_plotter, sorting_col="t")

# Make it servable as interactive app
pn.Column(viz).servable()

In [ ]:
# Start the Pathway pipeline execution in the background
%%capture --no-display
pw.run()